Use this file as a template to copy and build notebooks

In [ ]:
#@title Mount drive and load libraries
import os
from google.colab import drive

drive.mount('/content/drive/')
path = '/content/drive/MyDrive/msc_project'
os.chdir(path)

!pip install umap-learn[plot]

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from sklearn.utils.linear_assignment_ import linear_assignment
import umap

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/msc_project/all_paired/230722_cluster_tensor_scfv_10000.csv', low_memory=False)

In [ ]:
umap_embedding = umap.UMAP(n_components=10, random_state=42)
protein_umap = umap_embedding.fit_transform(test.iloc[:, 2:])
X = protein_umap[:, 0]
y = protein_umap[:, 1]
metadata = test['cluster']

new_df = pd.DataFrame({'X': X, 'y': y, 'metadata':metadata})

# Set scatter plot parameters for smaller points
scatter_kws = {'s': 10}  # Adjust the 's' value to control point size

if num_clus < 20:
  boolie = True
else:
  boolie = False

# Use the 'hue' argument to provide a factor variable
sns.lmplot(x='X', y='y', data=new_df, hue='metadata', fit_reg=False, legend=boolie, scatter_kws=scatter_kws)

#plt.scatter(new_df['X'], new_df['y'])

plt.show()